In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import linprog

# Generate random data for 75 coffee beans
np.random.seed(42)  # For reproducibility
data = {
    "Bean_ID": range(1, 76),
    "Origin": np.random.choice(["Colombia", "Brazil", "Ethiopia", "Kenya", "Uganda"], 75),
    "Aroma": np.random.randint(1, 11, size=75),
    "Acidity": np.random.randint(1, 11, size=75),
    "Price_per_kilo": np.random.randint(10, 51, size=75)  # Price range from $10 to $50
}
coffee_beans = pd.DataFrame(data)
coffee_beans['Price_Score'] = 10 - (coffee_beans['Price_per_kilo'] - 10) / 40 * 9  # Normalize and score price
coffee_beans['Total_Score'] = coffee_beans['Aroma'] + coffee_beans['Acidity'] + coffee_beans['Price_Score']

# Setup the linear programming problem
num_beans = len(coffee_beans)
c = -coffee_beans['Total_Score'].values  # Coefficients for the objective function (negative for maximization)
A_eq = np.ones((1, num_beans))  # Equality constraint to select exactly 7 beans
b_eq = [7]  # Exactly 7 beans must be selected

# Adding inequality constraints to ensure at least one bean from each origin is selected
origins = coffee_beans['Origin'].unique()
A_ub = -np.zeros((len(origins), num_beans))  # Initialize
b_ub = -np.ones(len(origins))  # At least one bean from each origin

for idx, origin in enumerate(origins):
    A_ub[idx, :] = -(coffee_beans['Origin'] == origin).astype(int)  # Create constraints for each origin

x_bounds = [(0, 1) for _ in range(num_beans)]  # Bounds for each variable (0 or 1 for binary decision variables)

# Solve the problem
result = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=x_bounds, method='highs')
selected_indices = result.x > 0.99  # Threshold due to floating-point precision

# Display the selected beans
selected_beans = coffee_beans[selected_indices]
selected_beans[['Bean_ID', 'Origin', 'Aroma', 'Acidity', 'Price_per_kilo', 'Total_Score']]


,Bean_ID,Origin,Aroma,Acidity,Price_per_kilo,Total_Score
5,6,Brazil,9,9,25,24.625
8,9,Ethiopia,8,3,10,21.000
10,11,Kenya,8,8,14,25.100
38,39,Colombia,5,10,25,21.625
42,43,Kenya,10,7,13,26.325
47,48,Uganda,10,5,17,23.425
67,68,Brazil,7,8,14,24.100
